 # Web data mining & semantics Project

### Load libraries and file pathes

In [1]:
pip install rdflib

Note: you may need to restart the kernel to use updated packages.


In [2]:
from rdflib import Graph, Namespace, RDF, RDFS, Literal, URIRef
from rdflib.plugins.sparql import prepareQuery
from rdflib.namespace import XSD
import numpy as np

In [16]:
# path to rdf file and ontology
rdf_file = 'Project.rdf'
mon_ns = Namespace('http://www.semanticweb.org/efrei/ontologies/2024/6/untitled-ontology-3#')

# load RDF file in a RDF graph 
g = Graph()
g.parse(rdf_file, format='xml')

<Graph identifier=N9f8a02a32f8244e39e45fb3bb3268d5c (<class 'rdflib.graph.Graph'>)>

### Question 1 - displays all the Persons

In [17]:
# Look for all instances of Client in the graph
clients = g.subjects(RDF.type, mon_ns.Client)

# Display Clients
print("Clients found in the RDF graph:")

for client in clients:
    print(client)

# Look for all instances of Deliveryman in the graph
deliverymen = g.subjects(RDF.type, mon_ns.Deliveryman)

# Display Deliveryman
print("\nDeliverymen found in the RDF graph:")

for deliveryman in deliverymen:
    print(deliveryman)


Clients found in the RDF graph:
http://www.semanticweb.org/efrei/ontologies/2024/6/untitled-ontology-3#Walid

Deliverymen found in the RDF graph:
http://www.semanticweb.org/efrei/ontologies/2024/6/untitled-ontology-3#Jenna
http://www.semanticweb.org/efrei/ontologies/2024/6/untitled-ontology-3#Pierre
http://www.semanticweb.org/efrei/ontologies/2024/6/untitled-ontology-3#Valentin


### Question 2 - displays all the Persons (using a query, without inference)

In [13]:
# Path to txt file
query_file = 'data/query_persons.txt'

# Load SPARQL query from file
with open(query_file, 'r', encoding='utf-8') as f:
    query_str = f.read()

# Prepae SPARQL query
query = prepareQuery(query_str)

# Execute query and get result
results = g.query(query)

# Display result
print("Clients and Deliverymen found in the RDF graph:")

for row in results:
    print(row.person)


Clients and Deliverymen found in the RDF graph:
http://www.semanticweb.org/efrei/ontologies/2024/6/untitled-ontology-3#Walid
http://www.semanticweb.org/efrei/ontologies/2024/6/untitled-ontology-3#Jenna
http://www.semanticweb.org/efrei/ontologies/2024/6/untitled-ontology-3#Pierre
http://www.semanticweb.org/efrei/ontologies/2024/6/untitled-ontology-3#Valentin


### Question 3 - Restaurants serving Italian food

In [6]:
# Look for all instances of Italian Restaurants in the graph
Italian_Restaurants = g.subjects(RDF.type, mon_ns.Italian)

# Display Italian Restaurants
print("Italian Restaurant found in the RDF graph:")

for Italian in Italian_Restaurants:
    print(Italian)


Italian Restaurant found in the RDF graph:
http://www.semanticweb.org/efrei/ontologies/2024/6/untitled-ontology-3#Pizza_Paris
http://www.semanticweb.org/efrei/ontologies/2024/6/untitled-ontology-3#Pizzeria_Roma


### Question 4 - Reads a name of a food and display infos

In [11]:
# function to get food uri 
def get_food_uri(food_name):
    for food in g.subjects (RDF.type, mon_ns.FoodProduct): 
        if (food, None, Literal (food_name)) in g:
            return food 
        else :
            return None

# Retrieve associated data for Food name
def get_food_details(food_uri, food_name):
    # Retrieve restaurants serving this food
    restaurants = g.objects(food_uri, mon_ns.hasRestaurant)
    print(f"\nRestaurants serving '{food_name}':")
    for restaurant in restaurants:
        print(restaurant)

    # Retrieve delivery services offering this food
    delivery_services = set(g.objects(food_uri, mon_ns.hasDeliveryService))
    print(f"\nDelivery services offering '{food_name}':")
    for delivery_service in delivery_services:
        print(delivery_service)

    # Retrieve offers for this food
    offers = set(g.objects(food_uri, mon_ns.hasOffer))
    print(f"\nOffers for '{food_name}':")
    for offer in offers:
        print(offer)
        # Retrieve availability details for each offer
        availability_ends = list(g.objects(offer, mon_ns.availabilityEnds))
        for availability_end in availability_ends:
            print(f" - Available until: {availability_end}")
            
    #Retrieve Location for this food
    locations = set(g.objects(food_uri, mon_ns.Location))
    print(f"\nLocation '{food_name}':")
    for location in locations:
        print(location)
        
# Use the function
food_name = input("Enter the name of a food: ")
food_uri = get_food_uri(food_name)
get_food_details(food_uri, food_name)

Enter the name of a food:  Pizza



Restaurants serving 'Pizza':
http://www.semanticweb.org/efrei/ontologies/2024/6/untitled-ontology-3#Pizza_Paris
http://www.semanticweb.org/efrei/ontologies/2024/6/untitled-ontology-3#Pizzeria_Roma

Delivery services offering 'Pizza':
http://www.semanticweb.org/efrei/ontologies/2024/6/untitled-ontology-3#Uber_It
http://www.semanticweb.org/efrei/ontologies/2024/6/untitled-ontology-3#Deliverou

Offers for 'Pizza':
http://www.semanticweb.org/efrei/ontologies/2024/6/untitled-ontology-3#SummerOffer_2024
http://www.semanticweb.org/efrei/ontologies/2024/6/untitled-ontology-3#Summer_Pizza

Location 'Pizza':
Lyon
Paris
Rome


### Question 5 - Displays all entities that are restaurants and delivery companies

In [15]:
# function to get subclasses of a class
def get_all_subclasses(graph, class_uri):
    subclasses = set()
    for subclass in graph.transitive_subjects(RDFS.subClassOf, class_uri):
        subclasses.add(subclass)
    return subclasses

# use function to get subclasses of Restaurant 
restaurant_subclasses = get_all_subclasses(g, mon_ns.Restaurant)

# Store all entities of type RestDelivery
rest_deliveries = set()
for subclass in restaurant_subclasses:
    for entity in g.subjects(RDF.type, subclass):
        rest_deliveries.add(entity)
        g.add((entity, RDF.type, mon_ns.RestDelivery))
        
# Get all entities of type DeliveryCompany
for entity in g.subjects(RDF.type, mon_ns.DeliveryCompany):
    rest_deliveries.add(entity)
    g.add((entity, RDF.type, mon_ns.RestDelivery))

# Define rules of format N3 to mark entities as RestDelivery
rule_content = """
{ ?entity a ex:Restaurant } => { ?entity a ex:RestDelivery } .
{ ?entity a ex:DeliveryCompany } => { ?entity a ex:RestDelivery } .
"""
# Write rules in file rest_delivery_rule.n3
with open("data/rest_delivery_rule.n3", "w") as file:
    file.write(rule_content)
    
# Displays entities of type RestDelivery 
print("RestDelivery entities:")
for rest_delivery in rest_deliveries:
    print(rest_delivery)

RestDelivery entities:
http://www.semanticweb.org/efrei/ontologies/2024/6/untitled-ontology-3#leafy_heaven
http://www.semanticweb.org/efrei/ontologies/2024/6/untitled-ontology-3#Deliverou
http://www.semanticweb.org/efrei/ontologies/2024/6/untitled-ontology-3#Pizzeria_Roma
http://www.semanticweb.org/efrei/ontologies/2024/6/untitled-ontology-3#Sushi_Paris
http://www.semanticweb.org/efrei/ontologies/2024/6/untitled-ontology-3#Uber_It
http://www.semanticweb.org/efrei/ontologies/2024/6/untitled-ontology-3#Botanical_Bistro
http://www.semanticweb.org/efrei/ontologies/2024/6/untitled-ontology-3#Pizza_Paris
http://www.semanticweb.org/efrei/ontologies/2024/6/untitled-ontology-3#Earthly_Delight


### Question 6 - Specifies 3 different rules and implement them in a python program

In [14]:
def get_all_subclasses(graph, class_uri):
    subclasses = set()
    for subclass in graph.transitive_subjects(RDFS.subClassOf, class_uri):
        subclasses.add(subclass)
    return subclasses
 
 
# Rule 1: Identify and list all instances Client class
client_instances = set()
for entity in g.subjects(RDF.type, mon_ns.Client):
    client_instances.add(entity)
    g.add((entity, RDF.type, mon_ns.Client))
 
# Rule 2: Extract and display all properties of Client "Walid"
walid_properties = {}
walid_instance = URIRef(mon_ns + "Walid")
for p, o in g.predicate_objects(subject=walid_instance):
    walid_properties[p] = o
 
# Rule 3: Find and list all subclasses of class Restaurant
restaurant_subclasses = get_all_subclasses(g, mon_ns.Restaurant)
 
rule_content = """
@prefix ex: <http://example.org/> .
@prefix mon: <http://www.semanticweb.org/efrei/ontologies/2024/6/untitled-ontology-3#> .
 
# Rule 1: Identify and list all instances of a specific class (Client)
{ ?entity a mon:Client } => { ?entity a mon:Client } .
 
# Rule 2: Extract and display all properties associated with a specific instance of a class
{ ?entity mon:hasPreference ?preference } => { ?entity mon:hasPreference ?preference } .
{ ?entity mon:Location ?location } => { ?entity mon:Location ?location } .
 
# Rule 3: Find and list all subclasses of a specific class (Restaurant)
{ ?subclass rdfs:subClassOf mon:Restaurant } => { ?subclass rdfs:subClassOf mon:Restaurant } .
"""
 
with open("data/rules.n3", "w") as file:
    file.write(rule_content)
 
 
print("Client Instances:")
for client in client_instances:
    print(client)
 
print("\nProperties of 'Walid':")
for prop, value in walid_properties.items():
    print(f"{prop}: {value}")
 
print("\nSubclasses of 'Restaurant':")
for subclass in restaurant_subclasses:
    print(subclass)

Client Instances:
http://www.semanticweb.org/efrei/ontologies/2024/6/untitled-ontology-3#Walid

Properties of 'Walid':
http://www.w3.org/1999/02/22-rdf-syntax-ns#type: http://www.semanticweb.org/efrei/ontologies/2024/6/untitled-ontology-3#Client
http://www.semanticweb.org/efrei/ontologies/2024/6/untitled-ontology-3#hasPreference: http://www.semanticweb.org/efrei/ontologies/2024/6/untitled-ontology-3#Pizzeria_Roma
http://www.semanticweb.org/efrei/ontologies/2024/6/untitled-ontology-3#Location: Lyon

Subclasses of 'Restaurant':
http://www.semanticweb.org/efrei/ontologies/2024/6/untitled-ontology-3#Vegetarian
http://www.semanticweb.org/efrei/ontologies/2024/6/untitled-ontology-3#Restaurant
http://www.semanticweb.org/efrei/ontologies/2024/6/untitled-ontology-3#Indian
http://www.semanticweb.org/efrei/ontologies/2024/6/untitled-ontology-3#Asian
http://www.semanticweb.org/efrei/ontologies/2024/6/untitled-ontology-3#Italian
http://www.semanticweb.org/efrei/ontologies/2024/6/untitled-ontology-3